# Alignment Error Visualization

This notebook collects COM data from the database and tries to quantify some alignment errors. The main results are shown in the plots at the end of the notebook.

In [ ]:
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd
from collections import OrderedDict
from IPython.display import HTML
from itertools import combinations

PIPELINE_ROOT = Path('./').absolute().parents[1]
PIPELINE_ROOT = PIPELINE_ROOT.as_posix() + '/src'
sys.path.append(PIPELINE_ROOT)
print(PIPELINE_ROOT)


In [ ]:
from library.controller.sql_controller import SqlController
from library.image_manipulation.filelocation_manager import FileLocationManager
from library.atlas.atlas_utilities import apply_affine_transform, get_affine_transformation, list_coms, \
    compute_affine_transformation
from library.registration.brain_structure_manager import BrainStructureManager
from library.registration.algorithm import umeyama



from library.utilities.utilities_process import M_UM_SCALE, SCALING_FACTOR, random_string, read_image, write_image


In [ ]:
animal = 'Allen'
um = 10
affine = False
debug = False
brainManager = BrainStructureManager(animal, um, affine, debug)

In [ ]:
structure = 'SC'
com = np.array([926, 183, 572])
brainManager.update_database_com(structure, com)

In [ ]:
def absolute_sum(l):
    la = np.array(l)
    nabs = (np.array(la*la))
    return np.sum(la, axis=0)

def sum_square_com(com):
    ss = np.sqrt(sum([s*s for s in com]))
    return ss

def generate_combinations(lst):
    """
    Generate all combinations of at least 3 elements from the given list.
    
    :param lst: List of elements
    :return: List of tuples containing the combinations
    """
    return list(combinations(lst, 5 ))

    #result = []
    #for r in range(3, len(lst) + 1):
    #    result.extend(combinations(lst, r))
    #return result

def get_umeyama(atlas_src, allen_src):

    A, t = umeyama(atlas_src.T, allen_src.T, with_scaling=True)
    transformation_matrix = np.hstack( [A, t ])
    transformation_matrix = np.vstack([transformation_matrix, np.array([0, 0, 0, 1])])

    return transformation_matrix

In [ ]:
atlas_all = list_coms('Atlas')
allen_all = list_coms('Allen')
common_keys = list(atlas_all.keys() & allen_all.keys())
atlas_common = np.array([atlas_all[s] for s in common_keys])
allen_common = np.array([allen_all[s] for s in common_keys])


In [ ]:
combinations_list = generate_combinations(common_keys)
print(len(combinations_list))

## Data Collection

In [ ]:
error = {}

for combo in combinations_list[:3000]:
    atlas_src = np.array([atlas_all[s] for s in combo])
    allen_src = np.array([allen_all[s] for s in combo])
    df_list = []
    sss = []
    matrix = compute_affine_transformation(atlas_src, allen_src)
    for structure in common_keys:
        atlas0 = np.array(atlas_all[structure])
        allen0 = np.array(allen_all[structure]) 
        transformed = apply_affine_transform(atlas0, matrix)
        transformed = [round(x,8) for x in transformed]
        difference = [round(a - b, 8) for a, b in zip(transformed, allen0)]
        ss = sum_square_com(difference)
        row = [structure, atlas0, allen0, transformed, difference, ss]
        df_list.append(row)
        sss.append(ss)
    error[combo] = sum(sss)



sorted_combos = {k: v for k, v in sorted(error.items(), key=lambda item: item[1])}
result = list(sorted_combos.items())[:5]
    


In [ ]:
result

In [ ]:
df_list = []
ckeys_single = ('IC', 'AP', 'LRt_R', '7N_L')
atlas_src = np.array([atlas_structures[s] for s in ckeys_single])
allen_src = np.array([allen_structures[s] for s in ckeys_single])
#matrix = get_umeyama(atlas_src, allen_src)
matrix = compute_affine_transformation(atlas_src, allen_src)
error = []
for structure in common_keys:
    atlas0 = np.array(atlas_all[structure])
    allen0 = np.array(allen_all[structure]) 
    transformed = apply_affine_transform(atlas0, matrix)
    transformed = [round(x,2) for x in transformed]
    difference = [round(a - b, 2) for a, b in zip(transformed, allen0)]
    ss = sum_square_com(difference)
    row = [structure, atlas0, allen0, transformed, difference, ss]
    df_list.append(row)
    error.append(ss)
print('SS', sum(error))

In [ ]:
columns = ['structure', 'atlas0', 'allen0', 'transformed', 'difference', 'sumsquares']
df = pd.DataFrame(df_list, columns=columns)
columns = ['atlas0', 'allen0', 'transformed', 'difference', 'sumsquares']
df.index.name = 'Index'
df.sort_values(by=['structure'], inplace=True)
HTML(df.to_html(index=False))

In [ ]:
sorted(common_keys)

In [ ]:
counter = 1
for L in range(3, 10):
    for subset in combinations(common_keys, L):
        counter += 1
print(counter)